# 8장 역참조 사용하기

In [17]:
import re

ex = '''<body>
<h1>Welcome to my Homepage</h1>
Content is divided into two sections:<br/>
<h2>SQL</h2>
Information about SQL.
<h2>RegEx</h2>
Information about Regular Expressions.
<h2>This is not valid HTML</h3>
</body>'''

p1 = re.compile(r'<[hH][1-6]>.*?<\/[hH][1-6]>')
res = p1.finditer(ex)
for r in res: print(r)

<re.Match object; span=(7, 38), match='<h1>Welcome to my Homepage</h1>'>
<re.Match object; span=(82, 94), match='<h2>SQL</h2>'>
<re.Match object; span=(118, 132), match='<h2>RegEx</h2>'>
<re.Match object; span=(172, 203), match='<h2>This is not valid HTML</h3>'>


## 역참조 이해하기
* p1에선 .\*(탐욕적 수량자)가 아니라 .\*?(게으른 수량자)를 사용한 것을 유의하자. 탐욕적 수량자를 썼으면 두 번째 줄의 ```<h1>부터 마지막 </h3>```까지 '가져왔을 수도' 있다.
* p1은 ```<h2>This is not valid HTML</h3>```처럼 잘못된 태그도 가져온다. 즉, 종료 태그가 일치할 때, 시작 태그가 어떤 텍스트인지 알 수가 없다는 것이다. 이럴 때 역참조를 사용한다.

## 역참조로 찾기

In [16]:
import re

ex = '''This is a block of of text,
several words here are are
repeated, and and they
should not be.'''

p1 = re.compile(r'[ ]+(\w+)[ ]+\1')

res = p1.finditer(ex)
for r in res: print(r)

<re.Match object; span=(15, 21), match=' of of'>
<re.Match object; span=(46, 54), match=' are are'>
<re.Match object; span=(64, 72), match=' and and'>


### 위 p1 살피기
* [ ]+ 공백이 하나 이상 연속되는 경우 일치
* \w+ 영숫자 문자 하나 이상 연속되는 경우 일치
* [ ]+ 그 뒤에 공백이 있을 때 일치
  
* 여기서 하위 표현식을 사용한 이유는, 반복을 위해서가 아니라 나중에 일치한 부분을 사용할 수 있도록 표시해놓은 것이다.
* \1은 앞서 일치한 하위 표현식을 참조하겠다는 의미이다. 즉 (\w+)와 일치한 문자는 \1과도 일치한다.
* 파이썬에서는 group이라는 이름의 배열이 포함된 결과 객체를 반환한다.

In [15]:
import re

ex = '''<body>
<h1>Welcome to my Homepage</h1>
Content is divided into two sections:<br/>
<h2>SQL</h2>
Information about SQL.
<h2>RegEx</h2>
Information about Regular Expressions.
<h2>This is not valid HTML</h3>
</body>'''

p1 = re.compile(r'<[hH]([1-6])>.*?<\/[hH]\1>')

res = p1.finditer(ex)
for r in res: print(r)

<re.Match object; span=(7, 38), match='<h1>Welcome to my Homepage</h1>'>
<re.Match object; span=(82, 94), match='<h2>SQL</h2>'>
<re.Match object; span=(118, 132), match='<h2>RegEx</h2>'>


* 위의 p1에선 \1으로 ([1-6])를 참조했다. 따라서 <[hH]([1-6])>와 <\/[hH]\1>는 일치한다.
* 역참조는 하위 표현식일 때만 동작한다.
* 참조하는 숫자는 1부터 시작하며, 0번째 참조는 표현식 전체를 의미한다.
* 숫자로 참조하는 경우는 여러 에러 사항이 있다. 이를 보완하고자, '이름 붙여 저장하는 기능'을 지원하는 정규 표현식 구현도 있다. 닷넷이 대표적이다.

## 치환 작업 수행

In [47]:
import re

ex = '''313-555-1234
248-555-9999
810-555-9000'''

print(re.sub(pattern = '(\d{3})(-)(\d{3})(-)(\d{4})',
            repl = r"(\1) \3-\5",
            string = ex))

(313) 555-1234
(248) 555-9999
(810) 555-9000


## 대소문자 변환
|Meta|Description|
|---|---|
|\E|\L or \U 변환의 끝|
|\l|소문자로 바꾸고 싶은 글자 앞에 사용|
|\L|\E를 만날 때까지 모든 문자 소문자로|
|\u|대문자로 바꾸고 싶은 글자 앞에 사용|
|\U|\E를 만날 때까지 모든 문자 대문자로|

In [81]:
import re

ex = '''<body>
<h1>Welcome to my Homepage</h1>
Content is divided into two sections:<br/>
<h2>SQL</h2>
Information about SQL.
<h2>RegEx</h2>
Information about Regular Expressions.
<h2>This is not valid HTML</h3>
</body>'''


print(re.sub(pattern = '(<[Hh]1>)(.*?)(<\/[Hh]1>)',
            repl = lambda x: x.group(1) + r'{}'.format(x.group(2).upper()) + x.group(3),
            string = ex))


<body>
<h1>WELCOME TO MY HOMEPAGE</h1>
Content is divided into two sections:<br/>
<h2>SQL</h2>
Information about SQL.
<h2>RegEx</h2>
Information about Regular Expressions.
<h2>This is not valid HTML</h3>
</body>


* Python에서는 \u, \U는 오로지 유니코드 패턴으로만 사용하므로, 아래 코드는 에러가 난다.  

`print(re.sub(pattern = '(<[Hh]1>)(.*?)(<\/[Hh]1>)',  
            repl = r"\1\U\2\E\3",  
            string = ex))  `